In [2]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time, datetime
import tweeterid

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)
    
# academic token - the token of IDC
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'

In [3]:
### General function that are used in many functions we built:

def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params, next_token = None, verbose = True):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    if verbose == True:
        print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def connect_to_endpoint_retweets(url, headers, params, next_token = None, verbose = True):
    params['pagination_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    if verbose == True:
        print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [4]:
def create_url_retweet_by_tweet_id(tweet_id, max_results = 10):

    #search_url = "https://api.twitter.com/1.1/statuses/retweets/:id.json" #get only the 100 last retweets of a given tweet id

    search_url = search_url.replace(":id", tweet_id)
    #change params based on the endpoint you are using
    query_params = {
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'max_results': max_results,
                    'next_token': {}}
    return (search_url, query_params)


########################################################### use:
#search_url, query_params = create_url_tweet_id(search_url, tweet_id)
#connect_to_endpoint(url, headers, params)

### function get retweets of a certain tweet id
The Following function enables getting all the retweets of a certain tweet

arguments:

+ **tweet_id** - The tweet id you wish to get its retweets
+ **max_results** - The max amount of retweets you wish to get
+ **next_token** - The token to look from

In [4]:
def create_url_retweets_by_tweet_id(tweet_id, max_results = 10, next_token = None):

    #search_url = "https://api.twitter.com/1.1/statuses/retweets/:id.json" #get only the 100 last retweets of a given tweet id
    search_url = "https://api.twitter.com/2/tweets/:id/retweeted_by"
    search_url = search_url.replace(":id", tweet_id)
    #change params based on the endpoint you are using
    query_params = {
                    'expansions': 'pinned_tweet_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    #'place.fields': 'contained_within, country, country_code, full_name, geo, id, name, place_type',
                    'max_results': max_results,
                    'pagination_token': {next_token}}
    return (search_url, query_params)


########################################################### use:
#search_url, query_params = create_url_tweet_id(search_url, tweet_id)
#connect_to_endpoint(url, headers, params)

In [6]:
headers = create_headers(os.environ['TOKEN'])

In [5]:
headers = create_headers(os.environ['TOKEN'])
search_url, query_params = create_url_retweets_by_tweet_id(tweet_id = "1473447665757310980") #"1475817472414650369"
json_response = connect_to_endpoint(search_url, headers, query_params) #new


Endpoint Response Code: 200



+ We want to create **a dir for each tweet_id** (for each conversation id) this dir will contain the tree of that converation id

Each such dir will contain the following:
- excel file with all the retweets 
- json file with all the retweets
- log file with retweet-tokens
- log file with the number of retweets read

#Function arguments explain:

+ **dir_name** -The dir name to store for each tweet-id all its retweets
+ **max_result** - check the maximum amount!!!

In [19]:
        #making dir to store, for each tweet-id all its retweets


In [7]:
def return_retweets_of_tweet_id_SMALL(tweet_id=None,
                                    max_results = 10, evaluate_last_token = False,
                                    limit_amount_of_returned_retweets = 10000000,
                                   verbose = False, dir_tree_name = "conversation_trees"):

    search_url = "https://api.twitter.com/2/tweets/:id/retweeted_by" #endpoint use to collect data from
    search_url = search_url.replace(":id", tweet_id)

    import os.path
    #making a dir for the tree - this file will cintain a unique file for each conversation id
    #dir_tree_name = "conversation_trees"
    try:
        os.mkdir(dir_tree_name)
        print("creating tree directory", dir_tree_name, "to store all the trees")
    except:
        print("The dir", dir_tree_name ,"already exist")
    
    
    #making dir (inside the tree dir) to store, for each tweet-id all its retweets
    name_for_tweet_id = "conv_tree_for_" + str(tweet_id)
    dir_name_for_tweet_id = os.path.join(dir_tree_name, name_for_tweet_id) 
    try:
        os.mkdir(dir_name_for_tweet_id)
        print("creating directory", dir_name_for_tweet_id, "to insert all the retweets of the given tweet-id")
    except:
        print("The dir", dir_name_for_tweet_id ,"already exist")

    ##### the log dir
    dir_log_name = os.path.join(dir_name_for_tweet_id, "log_retweets_for_tweet_id_" + tweet_id) 
    try:
        os.mkdir(dir_log_name)
        print("creating directory", dir_log_name, "to insert all the logs of the retweets for the tweet id - ", str(tweet_id))
    except:
        print("The dir", dir_log_name ,"already exist")
        
    ########################
#     path_for_log_dir_of_certain_user = os.path.join(dir_log_name, user_name)
#     try:
#         os.mkdir(path_for_log_dir_of_certain_user)
#         print("creating directory", path_for_log_dir_of_certain_user,"in the dir",dir_log_name, "to insert all the logs of the key opinion leader", user_name)
#     except:
#         print("The dir", path_for_log_dir_of_certain_user ,"already exist")

    path_for_dir_retriving_retweets_stream = os.path.join(dir_log_name, 'retriving_retweets_streem.txt')
    with open(path_for_dir_retriving_retweets_stream, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time + '\n\n')

    ########### If the token file exist already, then take the last token available, else start from token 1  ############ 
    tokens_location = os.path.join(dir_log_name, "tokens.txt") 

    if (evaluate_last_token == True and os.path.isfile(tokens_location) == True):
        a_file = open(tokens_location, "r")
        lines = a_file.readlines()
        last_lines = lines[-2]
        next_token = last_lines[0:-1]
        a_file.close()    
    else:
        next_token = None

    ################ Add a time stamp ########################################
    with open(tokens_location, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time+ '\n\n')

    ##########################################################################################

    continue_searching = True
    json_response_list = []
    next_tokens = []
    num_of_returned_retweets = 0
    counter_loops = 0

    while continue_searching == True and num_of_returned_retweets < limit_amount_of_returned_retweets:
        counter_loops +=1
        if counter_loops > 1:
            next_token = json_response["meta"]["next_token"]
            query_params["pagination_token"] = next_token
            print("token to insert:",next_token)
        #if the returned amount of retweets is getting close to the limit number, we need to alter the max_result,
        #so we won't get retweets beyond what we asked
        #if (limit_amount_of_returned_retweets - num_of_returned_retweets) < max_results:
        max_results = min(limit_amount_of_returned_retweets - num_of_returned_retweets,max_results)
        #else :
        #    max_results = max_results

    #change params based on the endpoint you are using
        query_params = {
                        'expansions': 'pinned_tweet_id',
                        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                        #'place.fields': 'contained_within, country, country_code, full_name, geo, id, name, place_type',
                        'max_results': max_results,
                        'pagination_token': next_token}

        
        json_response = connect_to_endpoint_retweets(headers=headers,url = search_url, params= query_params, next_token = next_token)

        json_response_list.append(json_response) #the first json_response itme
        num_of_returned_retweets += json_response["meta"]["result_count"]

        ##### making a dataframe out of the json response:
        try:
            a = pd.json_normalize(json_response["data"])
            a["id_new"] = "id: " + a["id"].astype("string")
            list_of_cols_to_add = ["id","id_new", "verified", "username", "name", "created_at", "public_metrics.followers_count",
                                   "public_metrics.following_count", "public_metrics.tweet_count",
                                   "public_metrics.listed_count", "pinned_tweet_id"]
            list_cols_to_drop = [x for x in a.columns if x not in list_of_cols_to_add]

            ##droping labels we don't need
            df_tweets_i = a.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

            for col in list_of_cols_to_add:
                if col not in df_tweets_i.columns:
                    df_tweets_i[col] = "NA"

            #sort columns by alphabetic order
            col_list_df_tweets_i = df_tweets_i.columns.tolist()
            col_list_df_tweets_i.sort()
            df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)

            name = tweet_id + ".csv"
            path_for_table = os.path.join(dir_name_for_tweet_id, name)
            if os.path.isfile(path_for_table) == False: #if this is the first table of tweets
                df_tweets_i.to_csv(path_for_table, index=True)
            else:
                df_tweets_i.to_csv(path_for_table, mode='a', index=True, header=False)
        except:
            print("no data / include in the json")

        path_for_dir_all_json_responses = os.path.join(dir_log_name, 'all_json_responses.json')
        with open(path_for_dir_all_json_responses, 'w') as outfile:
            json.dump(json_response_list, outfile)

        with open(path_for_dir_retriving_retweets_stream, 'a') as f:
            print_stat = str(counter_loops) + " -> Got from twitter " + str(json_response["meta"]["result_count"]) + " tweets, and there are more tweets of that user to get, I am bringing more tweets!"
            f.write(print_stat+'\n')
            print_total = "Total amount of tweets: " + str(num_of_returned_retweets)
            f.write(print_total+ '\n\n')

        if "next_token" in json_response["meta"]:
            if (verbose == True and counter_loops % 20 == 1):
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
            elif verbose == False:
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
            next_token = json_response["meta"]["next_token"]
            query_params["pagination_token"] = next_token
            next_tokens.append(next_token)
            #ids_token_print = "next token = " + next_token + "newest id: " + json_response["meta"]["newest_id"] + " | oldest id: " + json_response["meta"]["oldest_id"]
            ids_token_print = next_token
            with open(tokens_location, 'a') as f:
                f.write(ids_token_print + '\n\n')
        else:
            print("no more tweets from this user")
            continue_searching = False
            print("Total amount of collected tweets = ", num_of_returned_retweets)

        if num_of_returned_retweets >=limit_amount_of_returned_retweets:
            print("oooops, There may be more tweets to return, but you asked to limit the amount of returned tweets")
            print("infact you got", num_of_returned_retweets, "returned tweets and limited the function to get", limit_amount_of_returned_retweets, "tweets")

    #In what case we suspect that there may be more tweets that we didn't get? -->
    #When the number of tweets we asked to get is equal to the number of tweets we got back

    ### save all thr json responses in json file:

    return json_response_list, num_of_returned_retweets, next_tokens


In [8]:
tweet_id = "1473447665757310980"

json_response_list, num_of_returned_retweets, next_tokens = return_retweets_of_tweet_id_SMALL(tweet_id=tweet_id,
                                    max_results = 10, evaluate_last_token = False,
                                    limit_amount_of_returned_retweets = 100,
                                   verbose = True, dir_tree_name = "conversation_trees")

creating tree directory conversation_trees to store all the trees
creating directory conversation_trees/conv_tree_for_1473447665757310980 to insert all the retweets of the given tweet-id
creating directory conversation_trees/conv_tree_for_1473447665757310980/log_retweets_for_tweet_id_1473447665757310980 to insert all the logs of the retweets for the tweet id -  1473447665757310980
Endpoint Response Code: 200
1 Got from twitter 10 tweets, and there are more tweets of that user to get, I am bringing more tweets!

token to insert: 7140dibdnow9c7btw480y1pt2iosnkae791k6wk5880fo
Endpoint Response Code: 200
token to insert: 7140dibdnow9c7btw480y1pi734vtidfash9us1zan2uq
Endpoint Response Code: 200
token to insert: 7140dibdnow9c7btw480y1p7blvfiydf3nk53rrwhzxh3
Endpoint Response Code: 200
token to insert: 7140dibdnow9c7btw480y1olzu82wrdwf75yg3jbshq2e
Endpoint Response Code: 200
token to insert: 7140dibdnow9c7btw480y1ob5wnlh7cssxex09jyj020i
Endpoint Response Code: 200
token to insert: 7140dibdnow

In [55]:
def __return_retweets_of_tweet_id_SMALL(self, tweet_id=None,
                                    max_results = 10, evaluate_last_token = False,
                                    limit_amount_of_returned_retweets = 10000000,
                                   verbose = False, dir_tree_name = "conversation_trees"):

    search_url = "https://api.twitter.com/2/tweets/:id/retweeted_by" #endpoint use to collect data from
    search_url = search_url.replace(":id", tweet_id)

    import os.path
    #making a dir for the tree - this file will cintain a unique file for each conversation id
    #dir_tree_name = "conversation_trees"
    try:
        os.mkdir(dir_tree_name)
        print("creating tree directory", dir_tree_name, "to store all the trees")
    except:
        print("The dir", dir_tree_name ,"already exist")
    
    
    #making dir (inside the tree dir) to store, for each tweet-id all its retweets
    name_for_tweet_id = "conv_tree_for_" + str(tweet_id)
    dir_name_for_tweet_id = os.path.join(dir_tree_name, name_for_tweet_id) 
    try:
        os.mkdir(dir_name_for_tweet_id)
        print("creating directory", dir_name_for_tweet_id, "to insert all the retweets of the given tweet-id")
    except:
        print("The dir", dir_name_for_tweet_id ,"already exist")

    ##### the log dir
    dir_log_name = os.path.join(dir_name_for_tweet_id, "log_retweets_for_tweet_id_" + tweet_id) 
    try:
        os.mkdir(dir_log_name)
        print("creating directory", dir_log_name, "to insert all the logs of the retweets for the tweet id - ", str(tweet_id))
    except:
        print("The dir", dir_log_name ,"already exist")
        
    ########################
#     path_for_log_dir_of_certain_user = os.path.join(dir_log_name, user_name)
#     try:
#         os.mkdir(path_for_log_dir_of_certain_user)
#         print("creating directory", path_for_log_dir_of_certain_user,"in the dir",dir_log_name, "to insert all the logs of the key opinion leader", user_name)
#     except:
#         print("The dir", path_for_log_dir_of_certain_user ,"already exist")

    path_for_dir_retriving_retweets_stream = os.path.join(dir_log_name, 'retriving_retweets_streem.txt')
    with open(path_for_dir_retriving_retweets_stream, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time + '\n\n')

    ########### If the token file exist already, then take the last token available, else start from token 1  ############ 
    tokens_location = os.path.join(dir_log_name, "tokens.txt") 

    if (evaluate_last_token == True and os.path.isfile(tokens_location) == True):
        a_file = open(tokens_location, "r")
        lines = a_file.readlines()
        last_lines = lines[-2]
        next_token = last_lines[0:-1]
        a_file.close()    
    else:
        next_token = None

    ################ Add a time stamp ########################################
    with open(tokens_location, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time+ '\n\n')

    ##########################################################################################

    continue_searching = True
    json_response_list = []
    next_tokens = []
    num_of_returned_retweets = 0
    counter_loops = 0

    while continue_searching == True and num_of_returned_retweets < limit_amount_of_returned_retweets:
        counter_loops +=1
        if counter_loops > 1:
            next_token = json_response["meta"]["next_token"]
            query_params["pagination_token"] = next_token
            print("token to insert:",next_token)
        #if the returned amount of retweets is getting close to the limit number, we need to alter the max_result,
        #so we won't get retweets beyond what we asked
        if (limit_amount_of_returned_retweets - num_of_returned_retweets) < max_results:
            max_results = limit_amount_of_returned_retweets - num_of_returned_retweets
        else :
            max_results = max_results

    #change params based on the endpoint you are using
        query_params = {
                        'expansions': 'pinned_tweet_id',
                        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                        #'place.fields': 'contained_within, country, country_code, full_name, geo, id, name, place_type',
                        'max_results': max_results,
                        'pagination_token': next_token}

        
        json_response = self.__connect_to_endpoint_retweets(url = search_url, params= query_params, next_token = next_token)

        json_response_list.append(json_response) #the first json_response itme
        num_of_returned_retweets += json_response["meta"]["result_count"]

        ##### making a dataframe out of the json response:
        try:
            a = pd.json_normalize(json_response["data"])
            a["id_new"] = "id: " + a["id"].astype("string")
            list_of_cols_to_add = ["id","id_new", "verified", "username", "name", "created_at", "public_metrics.followers_count",
                                   "public_metrics.following_count", "public_metrics.tweet_count",
                                   "public_metrics.listed_count", "pinned_tweet_id"]
            list_cols_to_drop = [x for x in a.columns if x not in list_of_cols_to_add]

            ##droping labels we don't need
            df_tweets_i = a.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

            for col in list_of_cols_to_add:
                if col not in df_tweets_i.columns:
                    df_tweets_i[col] = "NA"

            #sort columns by alphabetic order
            col_list_df_tweets_i = df_tweets_i.columns.tolist()
            col_list_df_tweets_i.sort()
            df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)

            name = tweet_id + ".csv"
            path_for_table = os.path.join(dir_name_for_tweet_id, name)
            if os.path.isfile(path_for_table) == False: #if this is the first table of tweets
                df_tweets_i.to_csv(path_for_table, index=True)
            else:
                df_tweets_i.to_csv(path_for_table, mode='a', index=True, header=False)
        except:
            print("no data / include in the json")

        with open(path_for_dir_retriving_retweets_stream, 'a') as f:
            print_stat = str(counter_loops) + " -> Got from twitter " + str(json_response["meta"]["result_count"]) + " tweets, and there are more tweets of that user to get, I am bringing more tweets!"
            f.write(print_stat+'\n')
            print_total = "Total amount of tweets: " + str(num_of_returned_retweets)
            f.write(print_total+ '\n\n')

        if "next_token" in json_response["meta"]:
            if (verbose == True and counter_loops % 20 == 1):
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
            elif verbose == False:
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
            next_token = json_response["meta"]["next_token"]
            query_params["pagination_token"] = next_token
            next_tokens.append(next_token)
            #ids_token_print = "next token = " + next_token + "newest id: " + json_response["meta"]["newest_id"] + " | oldest id: " + json_response["meta"]["oldest_id"]
            ids_token_print = next_token
            with open(tokens_location, 'a') as f:
                f.write(ids_token_print + '\n\n')
        else:
            print("no more tweets from this user")
            continue_searching = False
            print("Total amount of collected tweets = ", num_of_returned_retweets)

        if num_of_returned_retweets >=limit_amount_of_returned_retweets:
            print("oooops, There may be more tweets to return, but you asked to limit the amount of returned tweets")
            print("infact you got", num_of_returned_retweets, "returned tweets and limited the function to get", limit_amount_of_returned_retweets, "tweets")

    #In what case we suspect that there may be more tweets that we didn't get? -->
    #When the number of tweets we asked to get is equal to the number of tweets we got back

    ### save all thr json responses in json file:
    path_for_dir_all_json_responses = os.path.join(dir_log_name, 'all_json_responses.json')
    with open(path_for_dir_all_json_responses, 'w') as outfile:
        json.dump(json_response_list, outfile)
    return json_response_list, num_of_returned_retweets, next_tokens


In [69]:
next_token

In [66]:
query_params["pagination_token"] = {next_token}

{None}

In [ ]:
    def __return_tweets_of_key_opinion_leader(self, query="", user_name=None,
                                        start_time = "2015-12-7T00:00:00Z",
                                        end_time = "2021-12-26T00:00:00Z",
                                        max_results = 10, evaluate_last_token = False,
                                        limit_amount_of_returned_tweets = 10000000,
                                       verbose_10 = False, dir_name = "key_opinion_leaders_tweets_tables_beta"
):
  
        search_url = "https://api.twitter.com/2/tweets/search/all" #endpoint use to collect data from

        import os.path
        try:
            os.mkdir(dir_name)
            print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
        except:
            print("The dir", dir_name ,"already exist")

        if user_name is not None:
            query = str(query) + " from:" + str(user_name)
            try:
                display_start_time = datetime.strptime(start_time.split("T")[0], "%Y-%m-%d").strftime("%d-%m-%Y")
                display_end_time = datetime.strptime(end_time.split("T")[0], "%Y-%m-%d").strftime("%d-%m-%Y")
            except:
                display_start_time = start_time
                display_end_time = end_time
            print("Bringing all the tweets of the user:", user_name, "from:", display_start_time, "to", display_end_time)
            print()

        ##### the log dir
        import os.path
        #dir_name = "key_opinion_leaders_tweets_tables_beta"
        dir_name_beta = "key_opinion_leaders_tweets_tables_beta"
        dir_log_name = os.path.join(dir_name_beta, "log_key_opinion_leaders") 

        # try:
        #     os.mkdir(dir_name)
        #     print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
        # except:
        #     print("The dir", dir_name ,"already exist")
        try:
            os.mkdir(dir_log_name)
            print("creating directory", dir_log_name, "to insert all the logs of the key opinion leaders")
        except:
            print("The dir", dir_log_name ,"already exist")
        ########################
        path_for_log_dir_of_certain_user = os.path.join(dir_log_name, user_name)
        try:
            os.mkdir(path_for_log_dir_of_certain_user)
            print("creating directory", path_for_log_dir_of_certain_user,"in the dir",dir_log_name, "to insert all the logs of the key opinion leader", user_name)
        except:
            print("The dir", path_for_log_dir_of_certain_user ,"already exist")
            
        path_for_dir_retriving_tweets_streem = os.path.join(path_for_log_dir_of_certain_user, 'retriving_tweets_streem.txt')
        with open(path_for_dir_retriving_tweets_streem, 'a') as f:
            from datetime import datetime
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
            current_time = "Current Time: " + current_time + "   *****************************************   "
            f.write(current_time + '\n\n')
            
        ########### If the token file exist already, then take the last token available, else start from token 1  ############ 
        tokens_location = os.path.join(dir_log_name, user_name, "tokens.txt") 

        if (evaluate_last_token == True and os.path.isfile(tokens_location) == True):
            a_file = open(tokens_location, "r")
            lines = a_file.readlines()
            last_lines = lines[-2]
            next_token = last_lines[0:-1]
            a_file.close()    
        else:
            next_token = None
            
        ################ Add a time stamp ########################################
        path_for_dir_tokens = os.path.join(path_for_log_dir_of_certain_user, 'tokens.txt')
        with open(path_for_dir_tokens, 'a') as f:
            from datetime import datetime
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
            current_time = "Current Time: " + current_time + "   *****************************************   "
            f.write(current_time+ '\n\n')
            
        ##########################################################################################

        continue_searching = True
        json_response_list = []
        next_tokens = []
        num_of_returned_tweets = 0
        counter_loops = 0
        
        while continue_searching == True and num_of_returned_tweets < limit_amount_of_returned_tweets:
            counter_loops +=1
            if counter_loops > 1:
                next_token = json_response["meta"]["next_token"]
                query_params["next_token"] = next_token
                print("token to insert:",next_token)
            #if the returned amount of tweets is getting close to the limit number, we need to alter the max_result,
            #so we won't get tweets beyond what we asked
            
            if (limit_amount_of_returned_tweets - num_of_returned_tweets) < max_results:
                max_results = limit_amount_of_returned_tweets - num_of_returned_tweets
                if max_results < 10:
                    max_results = 10
            else :
                max_results = max_results

            #change params based on the endpoint you are using
            query_params = {'query': query,
                        'start_time': start_time,
                        'end_time': end_time,
                        'max_results': max_results,
                        'expansions': 'author_id,in_reply_to_user_id,geo.place_id,entities.mentions.username,referenced_tweets.id',
                        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                        'next_token': {next_token}}
            
            json_response = self.__connect_to_endpoint(url = search_url,params= query_params, next_token = next_token)
            
            json_response_list.append(json_response) #the first json_response itme
            num_of_returned_tweets += json_response["meta"]["result_count"]

            ##### making a dataframe out of the json response:
            try:
                a = pd.json_normalize(json_response["data"])
                b = pd.json_normalize(json_response["includes"], ["users"]).add_prefix("users.")
                
                a.conversation_id = a.conversation_id.astype("string")
                a.id = a.id.astype("string")
                a["id_new"] = "id: " + a["id"].astype("string")
                a["conv_id_new"] = "conv_id: " + a["conversation_id"].astype("string")
            #c = pd.json_normalize(json_response["includes"]["places"]).add_prefix("places.")
                df_tweets_i = pd.merge(a, b, left_on="author_id", right_on="users.id")
                list_of_cols_to_add = ['author_id', 'conversation_id', "conv_id_new", "id", "id_new",
                                    'created_at','entities.mentions',
                            'public_metrics.like_count', 'public_metrics.quote_count',
                        'public_metrics.reply_count', 'public_metrics.retweet_count','referenced_tweets', 'text',
                            'users.created_at', 'users.description','users.id', 'users.name',
                        'users.public_metrics.followers_count', 'users.public_metrics.following_count',
                        'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
                        'users.username', 'users.verified']

                list_cols_to_drop = [x for x in df_tweets_i.columns if x not in list_of_cols_to_add]

                ##droping labels we don't need
                df_tweets_i = df_tweets_i.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

                for col in list_of_cols_to_add:
                    if col not in df_tweets_i.columns:
                        df_tweets_i[col] = "NA"

                #sort columns by alphabetic order
                col_list_df_tweets_i = df_tweets_i.columns.tolist()
                col_list_df_tweets_i.sort()
                df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)
                
                name = user_name + ".csv"
                path_for_table = os.path.join(dir_name_beta, name)
                if os.path.isfile(path_for_table) == False: #if this is the first table of tweets
                    df_tweets_i.to_csv(path_for_table, index=True)
                else:
                    df_tweets_i.to_csv(path_for_table, mode='a', index=True, header=False)
            except:
                print("no data / include in the json")
        
            with open(path_for_dir_retriving_tweets_streem, 'a') as f:
                print_stat = str(counter_loops) + " -> Got from twitter " + str(json_response["meta"]["result_count"]) + " tweets, and there are more tweets of that user to get, I am bringing more tweets!"
                f.write(print_stat+'\n')
                print_total = "Total amount of tweets: " + str(num_of_returned_tweets)
                f.write(print_total+ '\n\n')

            if "next_token" in json_response["meta"]:
                if (verbose_10 == True and counter_loops % 20 == 1):
                    print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
                elif verbose_10 == False:
                    print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!\n")
                next_token = json_response["meta"]["next_token"]
                query_params["next_token"] = next_token
                next_tokens.append(next_token)
                #ids_token_print = "next token = " + next_token + "newest id: " + json_response["meta"]["newest_id"] + " | oldest id: " + json_response["meta"]["oldest_id"]
                ids_token_print = next_token
                with open(path_for_dir_tokens, 'a') as f:
                    f.write(ids_token_print + '\n\n')
            else:
                print("no more tweets from this user")
                continue_searching = False
                print("Total amount of collected tweets = ", num_of_returned_tweets)
            
            if num_of_returned_tweets >=limit_amount_of_returned_tweets:
                print("oooops, There may be more tweets to return, but you asked to limit the amount of returned tweets")
                print("infact you got", num_of_returned_tweets, "returned tweets and limited the function to get", limit_amount_of_returned_tweets, "tweets")
        
        #In what case we suspect that there may be more tweets that we didn't get? -->
        #When the number of tweets we asked to get is equal to the number of tweets we got back
        
        ### save all thr json responses in json file:
        path_for_dir_all_json_responses = os.path.join(path_for_log_dir_of_certain_user, 'all_json_responses.json')
        with open(path_for_dir_all_json_responses, 'w') as outfile:
            json.dump(json_response_list, outfile)
        return json_response_list, num_of_returned_tweets, next_tokens


In [ ]:
def return_retweets_by_tweet_id_BIG(self, tweet_id="", dir_name="retweets", user_names =None, \
                                     max_results = 10,limit_amount_of_returned_retweets = 10000000, verbose = False):
    

                                    
                                    
                                    
                                    
                                    

In [ ]:
def return_retweets_by_tweet_id_BIG(self, query="", dir_name="tweets", user_names =None, \
        start_time = "2015-12-7T00:00:00Z", end_time = "2021-12-26T00:00:00Z",
        max_results = 10, evaluate_last_token = False, \
            limit_amount_of_returned_tweets = 10000000, verbose_10 = False):

            if type(user_names) != list:
                user_names = [user_names]
            
            #users_json_response_lists = []
            names_evaluated = []
            names_didnt_evaluated = []
            next_tokens_users= [] #this will include a list where eachelement is a list containing all the tokens off the specific user
            for name in user_names:
                print("Bringing tweets of", name)
                query = ""
                user_name = name
                try:

                    json_response_list, num_of_returned_tweets,next_tokens = self.__return_tweets_of_key_opinion_leader(query=query, user_name=user_name,
                                                            start_time = start_time,
                                                            end_time = end_time,
                                                            max_results = max_results,
                                                            limit_amount_of_returned_tweets = limit_amount_of_returned_tweets,
                                                                                                                verbose_10 = True)
                    print(num_of_returned_tweets)
                    if num_of_returned_tweets > 0:
                        names_evaluated.append(name)
                        next_tokens_users.append(next_tokens)
                

                    else:
                        names_didnt_evaluated.append(name)
                        print("The user:", name, "had", num_of_returned_tweets, "tweets!!")

                    print("---------------------------------------------------------------")
                except:
                    print("There was a problem with the key opinion leader:", name)
                    names_didnt_evaluated.append(name)
                    print("*************************************************************************************")

    import time, datetime,json, numpy as np
    from inputimeout import inputimeout, TimeoutOccurred


    def create_url_tweet_ids(self, search_url, tweet_ids_list, verbose = False):
    
        #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = search_url.replace("X", ','.join(tweet_ids_list))
        if verbose: print(search_url)
        #change params based on the endpoint you are using
        query_params = {
                        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                        'next_token': {}}
        return (search_url, query_params)



    #   sleep_time = 15*60 ## 15 minutes in sec
    #   tweet_ids = all_harvard_data["ID"].to_numpy().astype(str)

    from os import path


In [6]:
for i in json_response:
    print(i)

data
includes
errors
meta


In [7]:
json_response["data"]

[{'id': '1506200159604162562',
  'verified': False,
  'public_metrics': {'followers_count': 1,
   'following_count': 0,
   'tweet_count': 242,
   'listed_count': 0},
  'username': 'JjusticeBieber',
  'name': 'Justin Bieber',
  'created_at': '2022-03-22T09:24:53.000Z',
  'description': 'JUSTICE the new album out now'},
 {'pinned_tweet_id': '1405500123451301899',
  'id': '945189278849445888',
  'verified': False,
  'public_metrics': {'followers_count': 15,
   'following_count': 142,
   'tweet_count': 1615,
   'listed_count': 0},
  'username': 'iambellagella',
  'name': 'BELLA BRUCE GELLA',
  'created_at': '2017-12-25T07:07:39.000Z',
  'description': '🌺#annemarie @AnneMarie🌺 🏳️\u200d🌈#harrystyles @harry_styles 🏳️\u200d🌈❄️iG:IAMBELLAGELLA'},
 {'id': '4828270962',
  'verified': False,
  'public_metrics': {'followers_count': 57,
   'following_count': 33,
   'tweet_count': 773,
   'listed_count': 0},
  'username': 'GzlAranza',
  'name': 'Aranza Paredes',
  'created_at': '2016-01-29T02:14:23.0

In [9]:
#print("data:", json_response["data"])
#print("includes:", json_response["includes"])
print("errors:", json_response["errors"])
print("meta:", json_response["meta"])


errors: [{'value': '1496054215621369857', 'detail': 'Could not find tweet with pinned_tweet_id: [1496054215621369857].', 'title': 'Not Found Error', 'resource_type': 'tweet', 'parameter': 'pinned_tweet_id', 'resource_id': '1496054215621369857', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}, {'value': '784938598319607808', 'detail': 'Could not find tweet with pinned_tweet_id: [784938598319607808].', 'title': 'Not Found Error', 'resource_type': 'tweet', 'parameter': 'pinned_tweet_id', 'resource_id': '784938598319607808', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}, {'value': '1505012876813250562', 'detail': 'Could not find tweet with pinned_tweet_id: [1505012876813250562].', 'title': 'Not Found Error', 'resource_type': 'tweet', 'parameter': 'pinned_tweet_id', 'resource_id': '1505012876813250562', 'type': 'https://api.twitter.com/2/problems/resource-not-found'}]
meta: {'result_count': 10, 'next_token': '7140dibdnow9c7btw480y1pi5jvhk12c0zerx4c5jlhyq'

In [11]:
next_token = json_response["meta"]["next_token"] #here we extract the next token, we will use this token in a for loop
print(next_token)

7140dibdnow9c7btw480y1pi5jvhk12c0zerx4c5jlhyq


In [12]:
#### round 2:

headers = create_headers(os.environ['TOKEN'])
search_url, query_params = create_url_retweets_by_tweet_id(tweet_id = "1473447665757310980",
                                                          max_results = 10, next_token = next_token) #"1475817472414650369"
json_response = connect_to_endpoint(search_url, headers, query_params) #new


Endpoint Response Code: 200


In [15]:
print("meta:", json_response["meta"])


meta: {'result_count': 10, 'next_token': '7140dibdnow9c7btw480y1ownp3am92ahs30h4ihp7ozd', 'previous_token': '77qpymm88g5h9vqkluxceh7kd7q9mbibfzlfjka2vrsja'}


In [13]:
next_token = json_response["meta"]["next_token"]
print(next_token)

7140dibdnow9c7btw480y1ownp3am92ahs30h4ihp7ozd


In [169]:
print("ids:", json_response["ids"])
print("next_cursor:", json_response["next_cursor"])
print("next_cursor_str:", json_response["next_cursor_str"])
print("previous_cursor:", json_response["previous_cursor"])
print("previous_cursor_str:", json_response["previous_cursor_str"])
print("total_count:", json_response["total_count"])

ids: [1046342533897490433]
next_cursor: 0
next_cursor_str: 0
previous_cursor: 0
previous_cursor_str: 0
total_count: None


In [126]:
len(json_response["ids"])

81

In [14]:
search_url = "https://api.twitter.com/1.1/statuses/retweets/:id.json"
search_url = search_url.replace(":id", tweet_id)
search_url

'https://api.twitter.com/1.1/statuses/retweets/1475817472414650369.json'

In [17]:
json_response

[{'created_at': 'Sun Mar 20 14:37:54 +0000 2022',
  'id': 1505554226566635524,
  'id_str': '1505554226566635524',
  'text': 'RT @roymadpis789: Trying Twitter for the first time!😀 #fun #michael_kobaivanov',
  'truncated': False,
  'entities': {'hashtags': [{'text': 'fun', 'indices': [54, 58]},
    {'text': 'michael_kobaivanov', 'indices': [59, 78]}],
   'symbols': [],
   'user_mentions': [{'screen_name': 'roymadpis789',
     'name': 'Roy m',
     'id': 1460609632813170693,
     'id_str': '1460609632813170693',
     'indices': [3, 16]}],
   'urls': []},
  'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1460590577465147395,
   'id_str': '1460590577465147395',
   'name': 'Michael Kobaivanov',
   'screen_name': 'MichaelKobaiva1',
   'location': '',
   'descri

In [ ]:
1475817472414650369

In [5]:
def create_url_tweet_id(search_url, tweet_id):
    
    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
    search_url = search_url.replace(":id", tweet_id)
    #change params based on the endpoint you are using
    query_params = {
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


########################################################### use:
#search_url, query_params = create_url_tweet_id(search_url, tweet_id)
#connect_to_endpoint(url, headers, params)

In [6]:
#search url for idtweet - App rate limit: 300 requests per 15-minute window shared among all users of your app
#User rate limit (OAuth 1.0a): 900 requests per 15-minute window per each authenticated user
headers = create_headers(os.environ['TOKEN'])
search_url = "https://api.twitter.com/2/tweets/:id" #Change to the endpoint you want to collect data from
tweet_id = "1475817472414650369" #"1108157488581562373"
###-------------------------------------------------------------------------------------
search_url, query_params = create_url_tweet_id(search_url, tweet_id)
json_response = connect_to_endpoint(search_url, headers, query_params) #new

Endpoint Response Code: 200


In [7]:
json_response

{'data': {'lang': 'en',
  'public_metrics': {'retweet_count': 1,
   'reply_count': 1,
   'like_count': 1,
   'quote_count': 0},
  'reply_settings': 'everyone',
  'conversation_id': '1475817472414650369',
  'created_at': '2021-12-28T13:14:39.000Z',
  'author_id': '1460609632813170693',
  'source': 'Twitter Web App',
  'text': 'Trying Twitter for the first time!😀 #fun #michael_kobaivanov',
  'id': '1475817472414650369'},
 'includes': {'users': [{'username': 'roymadpis789',
    'id': '1460609632813170693',
    'name': 'Roy m',
    'public_metrics': {'followers_count': 2,
     'following_count': 7,
     'tweet_count': 1,
     'listed_count': 0},
    'created_at': '2021-11-16T14:04:28.000Z',
    'verified': False,
    'description': ''}]}}